# Kunskapskontroll

### Fyll i uppgifterna nedan innan du lämnar in på LearnPoint: 
Namn på samtliga gruppmedlemmar: 

Marcus Turesson, Pontus Paulsson, Jonathan Karlsson, Rasmus Hellgren, Tommy Wang

# Code

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split, GridSearchCV
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


In [2]:
# Below, set your own path where you have stored the data file. 
housing = pd.read_csv("housing.csv")

In [3]:
raw_df = pd.DataFrame(housing)

## EDA

In [4]:
raw_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [5]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [6]:
df = raw_df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20433 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  float64
 3   total_rooms         20433 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  float64
 6   households          20433 non-null  float64
 7   median_income       20433 non-null  float64
 8   median_house_value  20433 non-null  float64
 9   ocean_proximity     20433 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.7+ MB


Total_bedrooms is missing values so we are dropping those columns to clean the dataset

### Remove all houses where price is 500001

In [7]:
#less_than_500001 = df[df["median_house_value"] <= 500000]
# Check so that it worked
#less_than_500001[less_than_500001["median_house_value"] > 500000]

In [8]:
#df = less_than_500001
#df

### Pontus ger sig på något

In [9]:
# Kontrollera om det finns några kategoriska kolumner kvar
categorical_cols = housing.select_dtypes(include=['object']).columns
print(categorical_cols)

# Omvandla kategoriska variabler till numeriska med pd.get_dummies
housing = pd.get_dummies(housing, columns=categorical_cols)

Index(['ocean_proximity'], dtype='object')


In [10]:
# Fyll i saknade värden med medianvärden
housing = housing.fillna(housing.median())

# Kontrollera att alla saknade värden har fyllts i
missing_values = housing.isnull().sum()
print(missing_values[missing_values > 0])

Series([], dtype: int64)


In [ ]:
# Lägg till en binär variabel som indikerar om priset är begränsat
housing['capped'] = (housing['median_house_value'] == 500001).astype(int)

# Uppdatera features och target variabler
X = housing.drop('median_house_value', axis=1)
y = housing['median_house_value']

# Dela upp data i tränings- och testset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Träna om modellen med den nya variabeln
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)

# Utvärdera den nya modellens prestanda
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
rf_mae = mean_absolute_error(y_test, rf_y_pred)
rf_mse = mean_squared_error(y_test, rf_y_pred)
rf_r2 = r2_score(y_test, rf_y_pred)

print(f'MAE: {rf_mae}')
print(f'MSE: {rf_mse}')
print(f'R²: {rf_r2}')

# Undersök hur många hus som har priset 500001
capped_houses = housing[housing['median_house_value'] == 500001]
num_capped_houses = len(capped_houses)
total_houses = len(housing)

print(f"Antal hus med priset 500001: {num_capped_houses}")
print(f"Andel av totala datasetet: {num_capped_houses / total_houses * 100:.2f}%")


### View a graph of all houses

In [ ]:
plt.figure(figsize=(10, 10))

# Backgroud image
background_img = mpimg.imread('./pictures/california-map.jpg')
extent = [-124.48, -114.13, 32.53, 42.01]
scale_factor = 1.08  # Adjust the scale factor as needed for the desired enlargement
new_extent = [
    extent[0] - (extent[1] - extent[0]) * (scale_factor - 1) / 2,
    extent[1] + (extent[1] - extent[0]) * (scale_factor - 1) / 2,
    extent[2] - (extent[3] - extent[2]) * (scale_factor - 1) / 2,
    extent[3] + (extent[3] - extent[2]) * (scale_factor - 1) / 2
]
plt.imshow(background_img, extent=new_extent, aspect='auto')

# Scatter plot
scatter = plt.scatter(df['longitude'], df['latitude'], c=df['median_house_value'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Median House Value')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Geographical Distribution of Houses')
plt.show()

In [ ]:
# Histogrammen hjälper till att förstå fördelningen och spridningen av de numeriska variablerna i datasetet.

df.hist(bins=54, figsize=(20, 15)) # Skapar histogram för alla numeriska kolumner i df och en figurstorlek på 20x15.

for ax in plt.gcf().axes:
    ax.set_ylabel('Number of Houses')

plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
scatter = plt.scatter(df['median_income'], df['median_house_value'], 
                      c=df['ocean_proximity'].astype('category').cat.codes, # astype('category').cat.codes konverterar kategoriska värden till numeriska koder för färgning.
                      cmap='viridis', alpha=0.6) # Alpha är satt till 0.6 för att hantera överlappande punkter och göra plottet mer läsbart.
plt.colorbar(scatter, label='Ocean Proximity') # Genom att använda färg för att visa ocean_proximity kan vi se om närheten till havet påverkar husvärden.
plt.xlabel('Median Income (scaled * 10,000)')
plt.ylabel('Median House Value ($)')
plt.title('Median Income vs Median House Value')
plt.grid(True)

plt.show()

### Convert ocean_proximity to different dummy columns

In [ ]:
df = df.join(pd.get_dummies(df.ocean_proximity)).drop(["ocean_proximity"], axis=1)

### Heatmap av all correlation

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(df.corr(), annot=True, cmap="YlGnBu")

#### Vi ser här att median_house_value har rätt hög correlation med median_income

In [ ]:
#X = housing.drop(columns=['median_house_value'], axis=1)
#y = housing['median_house_value']
#X_train, X_test, y_train,  y_test = train_test_split(X,y, test_size=0.2, random_state=56)

In [ ]:
# Assuming df is your DataFrame
# Let's select the relevant columns for the features and target variable
X = df[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'population', 'median_income']]
y = df['median_house_value'] 

In [ ]:
X_train.head()

In [ ]:
X_train.describe()

In [ ]:
plt.figure(figsize=(5,5))
sns.pairplot(housing)
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("\nTraining data size:", X_train.shape)
print("Testing data size:", X_test.shape)

In [ ]:
# Fit the training data to a Linear Regression model
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

# Show your regression model's score
train_score = regression_model.score(X_train, y_train)
test_score = regression_model.score(X_test, y_test)
print(f'\nTraining score: {train_score}')
print(f'Testing score: {test_score}')

In [ ]:
# Get the coefficients and intercept of the trained regression model
coefficients = regression_model.coef_
intercept = regression_model.intercept_

# Select three samples from the testing set
samples_to_predict = X_test

# Manually calculate the predictions without using loops
predictions = np.dot(samples_to_predict.values, coefficients.reshape(-1, 1)).flatten() + intercept

# Get the actual data for the selected samples from the dataset
actual_data = y_test

# Print the predictions and actual data
for pred, actual in zip(predictions, actual_data):
    print("Prediction:", pred)
    print("Actual data:", actual)
    acc = regression_model.score(X_test, y_test)
    print(acc*100, '%')
    print()